In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
from tabulate import tabulate

In [ ]:
# model_id = "healio"
# model_id = "gpt-3.5-turbo"
q_types = ["yes_no", "factual", "list", "causal", "hypothetical", "complex"]
criteria = ["Grammar", "Coherence", "Context", "Correctness"]
models = ["healio", "gpt-3.5-turbo"]
scores = {}
for model_id in models:
    with open(f"evaluations_{model_id}.json", "r") as f:
        evaluations = json.load(f)
    scores[model_id] = {q_type: {
        "Grammar": 0.0,
        "Coherence": 0.0,
        "Context": 0.0,
        "Correctness": 0.0,
    } for q_type in q_types}
    for q in evaluations:
        e = q["evaluation"]
        if "/10" in e:
            e = e.strip().replace("\n", "").replace("/10", "/10 ").replace(".", " ").replace(":", " ").replace("(", " ").replace(")", " ").split(" ")
            e = [w for w in e if w != ""]
            for i, tok in enumerate(e):
                if "/10" in tok:
                    try:
                        nom = float(tok.split("/")[0])
                        denom = float(tok.split("/")[1])
                        scores[model_id][q["type"]][e[i-1]] += nom / denom
                    except Exception as ex:
                        continue  # unnecessary total grades
        else:
            pass
            # print("GRADING FAILED!")
            # for k, v in q.items():
            #     print(k, v)
    # print(model_id)
    # for k, v in scores.items():
    #     print(f"    {k} : {np.round(v/6, 2)}/10")
    # print("")

In [ ]:
for crit in criteria:
    print(crit)
    headers = ["q-type"] + models
    t = []
    avg = {m: 0.0 for m in models}
    for qt in q_types:
        row = [qt]
        for m in models:
            row.append(np.round(scores[m][qt][crit], 2))
            avg[m] += scores[m][qt][crit]
        t.append(row)
    t.append(["average"] + [np.round(avg[m]/6, 1) for m in models])
    print(tabulate(t, headers=headers))
    print(" ")
    print(" ")
    print(" ")